# Drive mounting e intalación de wdfb

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
# Cambiar el directorio de trabajo a la carpeta específica
os.chdir("/content/drive/MyDrive/Reconocimiento de Patrones")
!pip install wfdb

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.2 MB/s eta 0:00:00


# Importación de librerías y definición de funciones

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz, hilbert, welch, find_peaks, butter, filtfilt
from scipy.integrate import cumtrapz
from scipy.stats import linregress
from numpy.fft import fft, fftfreq
import pywt
import pickle

def butter_bandpass(lowcut, highcut, fs, order=5, filter_type = "band"):
    return butter(order, [lowcut, highcut], fs=fs, btype=filter_type)

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5, filter_tp = "band"):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order, filter_type = filter_tp)
    w, h = freqz(b, a, fs=fs, worN=2000)
    y = lfilter(b, a, data)
    return y,w,h

def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def fft(x,fs = None,n = None):
    X = np.fft.fftshift(np.fft.fft(x,n))
    N = len(X)
    k = np.arange(N)
    w = 2*np.pi*k/N
    w = np.fft.fftshift(w)
    w = np.unwrap(w - 2*np.pi)
    if fs != None:
        f = w*fs/(2*np.pi)
        return X,f,w
    else:
        return X,w

def extraer_nombres_archivos_dat(directorio):
    nombres_archivos_dat = []
    for raiz, dirs, archivos in os.walk(directorio):
        for archivo in archivos:
            if archivo.endswith('.dat'):
                nombres_archivos_dat.append(archivo[:-4])
    return nombres_archivos_dat

def normalize_data(data):
    normalized_data = []
    for sequence in data:
        mean = np.mean(sequence)
        std = np.std(sequence)
        if std != 0:
            normalized_sequence = (sequence - mean) / std
        else:
            normalized_sequence = sequence - mean
        normalized_data.append(normalized_sequence)
    return normalized_data

# Preprocessing data

In [3]:
file_path = './dataset/waves/'

names = extraer_nombres_archivos_dat(file_path)

import wfdb
import matplotlib.pyplot as plt
import numpy as np


segm_dict = {"ABP":[],"CBFV": [] ,"ICP":[]}

for i, record_name in enumerate(names):
    # print(record_name)
    path = './dataset/waves/'
    record = wfdb.rdrecord(record_name=path+record_name)
    signal = record.p_signal
    cont = record.__dict__
    titles = cont["sig_name"]
    sig = cont["p_signal"]
    fs = cont["fs"]
    units = cont["units"]
    adc_gain = cont["adc_gain"]
    sig_l = cont["sig_len"]

    adj_sig = sig
    sig_dict = {titles[i]:adj_sig[:,i] for i in range(0,3)}
    units_dict = {titles[i]:units[i] for i in range(0,3)}

    filt_dict = {}
    for key, value in sig_dict.items():
        sig = value
        filt_sig,_,_ = butter_bandpass_filter(sig, 7.0, 25.0, fs, order=6, filter_tp = 'stop')
        filt_dict[key] = filt_sig

    fft_dict = {}
    for key, value in filt_dict.items():
        sig = value
        X,f,w = fft(sig, fs)
        fft_dict[key] = X
    t_seg = 5
    N_seg = 5*fs


    # Aplicar filtro
    filtered_abp = butter_lowpass_filter(filt_dict["ABP"], cutoff=10, fs=125, order=2)

    # Detección de picos
    peaks, _ = find_peaks(filtered_abp, distance=50)  # 'distance' depende de la frecuencia cardiaca promedio

    for i, idx in enumerate(peaks[:-1]):
        seg_abp = filt_dict["ABP"][peaks[i]:peaks[i+1]]
        seg_cbfv = filt_dict["CBFV"][peaks[i]:peaks[i+1]]
        seg_icp = filt_dict["ICP"][peaks[i]:peaks[i+1]]
        segm_dict["ABP"].append(seg_abp)
        segm_dict["CBFV"].append(seg_cbfv)
        segm_dict["ICP"].append(seg_icp)

norm_segm_dict = {}

norm_segm_dict['ABP'] = normalize_data(segm_dict['ABP'])
norm_segm_dict['CBFV'] = normalize_data(segm_dict['CBFV'])
norm_segm_dict['ICP'] = normalize_data(segm_dict['ICP'])

# Export data

In [4]:
with open('preprocessed_data.pickle', 'wb') as file:
    pickle.dump(filt_dict, file)
    pickle.dump(segm_dict, file)
    pickle.dump(norm_segm_dict, file)